<a href="https://colab.research.google.com/github/OleksiiLatypov/Regression_of_Used_Car_Prices/blob/main/raw_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e9:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76728%2F9057646%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240914%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240914T183327Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D166a40885b24016e46b7a3b3f80093cee320d3018e4f19513746601d57353b7154b7d8308371d950ff9ddfa288d0ef63832f60b1de2e2482a58aac7c50ef6a9bc700386621c95904cd3b6fd8b5f79b5e68402bd78a7cc8a858b0d1ab646a6815c46833c741776cbfd9486b8cfdd896e978a261bc2c74a14ec5fa3afbd8ccf11b4dc9a113024fdb3da683b7d7f4949e9fe76b422f137996099730af4d0a8e97963515a996d9a7b483c0a614cb7a5e2ab4a172b9e7f3d103a45bb483e2fb5fe58fb5ff774aa1b8ed2bf41b3514c09e2d69bbc76cc24e82de809c3bb13420d05e8d02f6e974b0ab4962f9c3c505e9dbb30b60077f7c84b1bb021269a06bdc71f1a5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import matplotlib.pyplot as plt
import re
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge, SGDRegressor, LinearRegression
import category_encoders as ce


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 17.3+ MB


In [ ]:
df = pd.concat([train, test], axis=0, ignore_index=True)

In [ ]:
to_replace = []
for row in df.loc[df['fuel_type'] == 'not supported', 'engine']:
    fuel = re.search(r'(Gasoline|Diesel|Hybrid|Flex Fuel|Electric|Plug-In Hybrid)', row)
    if fuel.group(1) == 'Flex Fuel':
        to_replace.append('E85 Flex Fuel')
    else:
        to_replace.append(fuel.group(1))
print(to_replace)

['E85 Flex Fuel', 'Gasoline', 'Electric', 'Gasoline', 'Electric', 'Gasoline', 'Electric', 'Gasoline', 'Gasoline', 'Electric', 'Gasoline', 'Electric', 'Gasoline', 'Gasoline', 'Electric', 'Electric', 'Gasoline', 'Gasoline', 'Electric', 'Electric', 'Electric', 'Gasoline', 'E85 Flex Fuel', 'Gasoline', 'Electric', 'Electric', 'Gasoline', 'Gasoline', 'Electric']


In [ ]:
df.loc[df['fuel_type'] == 'not supported', 'fuel_type'] = to_replace

In [ ]:
df.loc[df['fuel_type'] == 'not supported', 'fuel_type']

Series([], Name: fuel_type, dtype: object)

In [ ]:
fuel = df[pd.isna(df['fuel_type'])]
type_of_fuel = []
for col in fuel['engine']:
    #print(col)
    if 'Hybrid' in col:
        type_of_fuel.append('Hybrid')
        #print('hybrid')
    elif 'Gasoline' in col:
        type_of_fuel.append('Gasoline')
        #print('gas')
    elif 'Flex Fuel' in col:
        type_of_fuel.append('E85 Flex Fuel')
        #print('flex')
    elif 'Diesel' in col:
        type_of_fuel.append('Diesel')
        #print('diesel')
    else:
        type_of_fuel.append('Electric')

print(len(type_of_fuel))

nan_indices = df[pd.isna(df['fuel_type'])].index
df.loc[nan_indices, 'fuel_type'] = type_of_fuel

8466


In [ ]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])

In [ ]:
df = df.drop('clean_title', axis=1)

In [ ]:
train_df = df.iloc[:188533, :]
test_df = df.iloc[188533:, :]

In [ ]:
X = train_df.copy()
y = X['price']
X = X.drop('price', axis=1)

In [ ]:
X.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported


In [ ]:
cbe_encoder = ce.cat_boost.CatBoostEncoder(a=1)
X_encoded = cbe_encoder.fit_transform(X, y)

In [ ]:
import category_encoders as ce
encoder = ce.GLMMEncoder(cols=['brand', 'model', 'fuel_type', 'engine',
                               'transmission', 'int_col', 'ext_col', 'accident'])
X_glmm_encoded = encoder.fit_transform(X, y)


In [ ]:
X_glmm_encoded.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident
0,-31244.785164,-29205.079333,2007,213000,-1809.404331,-31657.554628,-34167.721835,-24246.095719,-44959.020791,11894.218182
1,-17807.856371,-24151.031641,2002,143250,-1809.404331,-29497.859697,-34167.721835,-35800.114683,-43188.553600,-11894.219815
2,-6719.512470,-16787.529710,2002,136731,-18283.176231,-25001.535779,-34167.721835,-27020.589667,-44959.020791,11894.218182
3,4971.451407,6111.128240,2017,19500,-1809.404331,-1249.491877,-15677.382584,-25265.632669,-27623.124415,11894.218182
4,2685.285407,22236.092514,2021,7388,-1809.404331,-5760.451770,-18752.307564,-25265.632669,-43188.553600,11894.218182


In [ ]:
X_encoded

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident
0,43878.016178,43878.016178,2007,213000,43878.016178,43878.016178,43878.016178,43878.016178,43878.016178,43878.016178
1,43878.016178,43878.016178,2002,143250,24039.008089,43878.016178,24039.008089,43878.016178,43878.016178,43878.016178
2,43878.016178,43878.016178,2002,136731,43878.016178,43878.016178,17692.338726,43878.016178,24039.008089,24039.008089
3,43878.016178,43878.016178,2017,19500,17692.338726,43878.016178,43878.016178,43878.016178,43878.016178,20659.338726
4,43878.016178,43878.016178,2021,7388,24519.254044,43878.016178,43878.016178,44439.008089,24438.508089,26744.504044
...,...,...,...,...,...,...,...,...,...,...
188528,45662.598848,51082.622331,2017,49000,43362.201639,54490.416408,50203.767634,40803.380421,29954.666055,49122.624457
188529,51243.263576,56311.490891,2018,28600,43362.106075,55916.222764,56443.420878,40803.076781,45523.650380,25334.405346
188530,51242.155540,63688.241658,2021,13650,43362.025574,73399.047757,47119.639879,40802.830214,45523.506204,49122.477328
188531,41001.089007,37351.315850,2022,13895,43362.287870,63050.155915,69484.369907,99767.809378,45523.890483,49122.734379


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from catboost import CatBoostRegressor


clf_catboost = CatBoostRegressor(iterations=300,
                                  learning_rate=0.02,
                                  depth=7,
                                 l2_leaf_reg=2,
                                # border_count=100,
                                 verbose=0)  # verbose=0 to suppress output during cross-validation


# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation and evaluate RMSE
rmse_scores = -cross_val_score(clf_catboost, X_glmm_encoded, y, scoring='neg_root_mean_squared_error', cv=kf)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")

clf_catboost.fit(X_glmm_encoded, y)

Mean RMSE: 72029.9675
Standard Deviation of RMSE: 3661.5313


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from catboost import CatBoostRegressor


clf_catboost = CatBoostRegressor(iterations=275,
                                 learning_rate=0.03,
                                 depth=7,
                                 l2_leaf_reg=3,
                                # border_count=100,
                                 verbose=0)  # verbose=0 to suppress output during cross-validation


# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation and evaluate RMSE
rmse_scores = -cross_val_score(clf_catboost, X_encoded, y, scoring='neg_root_mean_squared_error', cv=kf)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")

clf_catboost.fit(X_encoded, y)

Mean RMSE: 72787.3706
Standard Deviation of RMSE: 3689.9015


72795 depth7

72807 depth8

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

# Define the parameter grid
param_grid = {
    'iterations': [200, 300, 400],                # Number of trees
    'learning_rate': [0.01, 0.03, 0.1],           # Learning rate
    'depth': [6, 7, 8],                           # Depth of trees
    'l2_leaf_reg': [1, 3, 5],                     # L2 regularization
    'border_count': [128, 254]                    # Number of discrete values for categorical features
}

# Initialize CatBoostRegressor
model = CatBoostRegressor(verbose=0)  # Turn off logging

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # Use RMSE as the scoring metric
    cv=5,  # Number of folds in cross-validation
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_encoded, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert negative RMSE to positive

print("Best Parameters:", best_params)
print("Best RMSE:", best_score)


In [ ]:
test_df = test_df.drop('price', axis=1)

In [ ]:
test_X_encoded = cbe_encoder.transform(test_df)

In [ ]:
test_X_encoded

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident
188533,53203.900904,20058.765023,2015,98000,43362.451557,20188.320019,25865.658244,40803.612790,29954.565848,49122.840862
188534,53203.900904,74093.338315,2020,9142,55477.453445,77381.065695,56442.140076,31879.111321,45524.102670,49122.840862
188535,40512.115294,50409.071842,2022,28121,43362.451557,56136.368430,63983.704271,40803.612790,49467.149954,49122.840862
188536,41005.120868,28170.680579,2016,61258,43362.451557,47609.679241,59587.105969,87287.295069,45524.102670,49122.840862
188537,41005.120868,24805.152611,2018,59000,43362.451557,30609.074948,31712.222924,47908.807742,45524.102670,49122.840862
...,...,...,...,...,...,...,...,...,...,...
314218,51244.015335,35601.178520,2014,83315,43362.451557,47428.598343,47123.215642,42417.541462,45524.102670,49122.840862
314219,41005.120868,57151.104279,2019,29336,43362.451557,60148.862884,59587.105969,40803.612790,45524.102670,49122.840862
314220,41005.120868,29456.968655,2012,77634,43362.451557,26681.165921,31712.222924,42417.541462,45524.102670,49122.840862
314221,41005.120868,34428.223121,2012,112000,43362.451557,26681.165921,31712.222924,42417.541462,45524.102670,49122.840862


In [ ]:
test_id = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
predictions = clf_catboost.predict(test_X_encoded)
test_id['price'] = predictions
test_id.to_csv('submission_catboost_1.csv', index=False)

In [ ]:
test_id.head(10)

,id,price
0,188533,17857.738751
1,188534,75021.669290
2,188535,56729.277552
3,188536,33511.735326
4,188537,32236.218115
5,188538,15636.311395
6,188539,13144.278618
7,188540,51160.070443
8,188541,63515.107894
9,188542,34495.890348
